In [1]:
import glob

import os
from pathlib import Path

In [2]:
path = Path(os.getcwd())
root = Path(path.parent.absolute())

model_path = root / 'Models' / 'Pretrained Inception'

In [3]:
import tensorflow as tf
import numpy as np
import gc

def train_LSTM_model(path, model_name, num_batches, epochs, batch_size, validation_split = 0.1):
    # I am not sure that callbacks work in this context, further testing must be done.
    model = tf.keras.models.load_model(path / model_name)
    model.summary()
    
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    history = []
    for epoch in range(epochs):
        print('---- EPOCH ' + str(epoch) + ' ----')
        for batch in range(num_batches):
            features = np.load(path / ('training_triples/features_batch_' + str(batch) + '.npy'), allow_pickle = True)
            captions = np.load(path / ('training_triples/captions_batch_' + str(batch) + '.npy'), allow_pickle = True)
            outputs = np.load(path / ('training_triples/outputs_batch_' + str(batch) + '.npy'), allow_pickle = True)

            history.append(model.fit(x = [features, captions], y = outputs, epochs=1, batch_size = batch_size, verbose=1, validation_split = validation_split, callbacks = [early_stop]))
            
            gc.collect()
            tf.keras.backend.clear_session()
        
        model.save(path / 'TrainedLSTM' / ('epoch=' + str(epoch)))
    
    return model, np.array(history)

In [4]:
batches = glob.glob(str(model_path) + '/training_triples/*')
num_batches_training = int(len(batches)/3)
num_batches_training

12

In [5]:
BATCH_SIZE = 20 # It might be better to use max_caption_len for this batch size. More experimentation should be carried out.
EPOCHS = 2
trained_LSTM_model, history_arr = train_LSTM_model(model_path, 'UntrainedLSTM', num_batches_training, EPOCHS, BATCH_SIZE, 0.1)
#np.save(model_path / 'HistoryArray', history_arr, allow_pickle = True)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
captions (InputLayer)           [(None, 341)]        0                                            
__________________________________________________________________________________________________
features (InputLayer)           [(None, 2048)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 341, 200)     894200      captions[0][0]                   
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 2048)         0           features[0][0]                   
____________________________________________________________________________________________

INFO:tensorflow:Assets written to: D:\Mestrado\Applications of Deep Learning\Pull Request - Rename\Models\Pretrained Inception\TrainedLSTM\epoch=0\assets


INFO:tensorflow:Assets written to: D:\Mestrado\Applications of Deep Learning\Pull Request - Rename\Models\Pretrained Inception\TrainedLSTM\epoch=0\assets
C:\Users\nicol\Anaconda3\envs\tf-gpu\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


---- EPOCH 1 ----
2/2 [==============================] - 2s 461ms/step - loss: 5.6788 - accuracy: 0.0370 - val_loss: 8.5001 - val_accuracy: 0.0000e+00


INFO:tensorflow:Assets written to: D:\Mestrado\Applications of Deep Learning\Pull Request - Rename\Models\Pretrained Inception\TrainedLSTM\epoch=1\assets


INFO:tensorflow:Assets written to: D:\Mestrado\Applications of Deep Learning\Pull Request - Rename\Models\Pretrained Inception\TrainedLSTM\epoch=1\assets
C:\Users\nicol\Anaconda3\envs\tf-gpu\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


AttributeError: Can't pickle local object 'Loader._recreate_base_user_object.<locals>._UserObject'

In [11]:
np.save(model_path / 'HistoryArray.npy', history_arr, allow_pickle = True)

AttributeError: 'numpy.ndarray' object has no attribute 'history'

In [6]:
history_arr

array([<keras.callbacks.History object at 0x00000220F0BC2EB0>,
      dtype=object)